In [1]:
import os
import numpy as np

The values for the fragmentation functions (FF) are taken from the publication:
https://journals.aps.org/prd/abstract/10.1103/PhysRevD.105.L011103
The branching ratios for the decays into electrons are taken from the 2020 ed. of the PDG booklet.
For most charmed hadrons inclusive branching ratios are provided, however in the case of $\Xi_{c}^{0}$ and $\Xi_{c}^{+}$ only one deacy in electrons is listed and no inclusive value is given.

The values are stored in a nested list.
Each sublist is contains: `name, FF,statFF,syst up, syst down, BR, unc up, unc down`

In [2]:
valuesHadrons = [["$D^{0}$",  39.1 ,1.7, 2.5, 3.7,  6.49, 0.11, 0.11],
["$D^{+}$",                   17.3 ,1.8, 1.7, 2.1, 16.07, 0.30, 0.30],
["$D^{+}_{\mathrm s}$",       7.3  ,1.0, 1.9, 1.1,  6.50, 0.40, 0.40],
["$\Lambda^{+}_{\mathrm c}$", 20.4 ,1.3, 1.6, 2.2,  3.95, 0.35, 0.35],
["$\Xi^{0}_{\mathrm c}$",     8.0  ,1.2, 2.5, 2.4,  1.80, 1.20, 1.20],
["$\Xi^{+}_{\mathrm c}$",     8.0  ,1.2, 2.5, 2.4,  7.00, 4.00, 4.0]] # update in the 2021 pdg
# ["$\Xi^{+}_{\mathrm c}$",     8.0  ,1.2, 2.5, 2.4,  2.30, 0.70, 0.80]]

The central value is simply the product of the fragmentation function which is `[1]` and the branching ratio `[5]`.
The uncertainties then follow:
* [2] : statistical uncertainty of FF
* [3] : upper systematic uncertainty of FF
* [4] : lower systematic uncertainty of FF
* [6] : upper uncertainty of BR (from PDG)
* [7] : upper uncertainty of BR (from PDG)
To make the further calculations easier, we also add the stat. and syst. in the FF case.

In [3]:
def quadSum (A,B):
    return np.sqrt(A**2 + B**2)


lst_values = [[] for _ in range(len(valuesHadrons))]

for i in range(len(valuesHadrons)):
    # add the hadrin name
    lst_values[i].append(valuesHadrons[i][0])
    # the first value:
    lst_values[i].append(valuesHadrons[i][1])
    # calculate uncertaities on first value    
    lst_values[i].append(quadSum(valuesHadrons[i][2],valuesHadrons[i][3]))
    # calculate uncertaities on second value  
    lst_values[i].append(quadSum(valuesHadrons[i][2],valuesHadrons[i][4]))
    # and wirte the last 3
    lst_values[i].append(valuesHadrons[i][5])
    lst_values[i].append(valuesHadrons[i][6])
    lst_values[i].append(valuesHadrons[i][7])
for i in range(len(lst_values)):
    print(lst_values[i])

['$D^{0}$', 39.1, 3.023243291566195, 4.071854614300467, 6.49, 0.11, 0.11]
['$D^{+}$', 17.3, 2.4758836806279896, 2.7658633371878665, 16.07, 0.3, 0.3]
['$D^{+}_{\\mathrm s}$', 7.3, 2.1470910553583886, 1.4866068747318506, 6.5, 0.4, 0.4]
['$\\Lambda^{+}_{\\mathrm c}$', 20.4, 2.0615528128088303, 2.5553864678361276, 3.95, 0.35, 0.35]
['$\\Xi^{0}_{\\mathrm c}$', 8.0, 2.773084924772409, 2.6832815729997477, 1.8, 1.2, 1.2]
['$\\Xi^{+}_{\\mathrm c}$', 8.0, 2.773084924772409, 2.6832815729997477, 7.0, 4.0, 4.0]


With this we can simplify the list and the make the calculations a bit more readable.

In [4]:
valuesHadrons2 = [["$D^{0}$", 39.1 ,3.0232, 4.0719,  6.49, 0.11, 0.11],
["$D^{+}$",                   17.3 ,2.4759, 2.7659, 16.07, 0.30, 0.30],
["$D^{+}_{\mathrm s}$",       7.3  ,2.1471, 1.4866,  6.50, 0.40, 0.40],
["$\Lambda^{+}_{\mathrm c}$", 20.4 ,2.0616, 2.5554,  3.95, 0.35, 0.35],
["$\Xi^{0}_{\mathrm c}$",     8.0  ,2.7731, 2.6833,  1.80, 1.20, 1.20],
["$\Xi^{+}_{\mathrm c}$",     8.0  ,2.7731, 2.6833,  7.00, 4.00, 4.0]] # update in the 2021 pdg
# ["$\Xi^{+}_{\mathrm c}$",     8.0  ,2.7731, 2.6833,  2.30, 0.70, 0.80]]

In [5]:
valuesHadrons2 = lst_values

In [6]:
def calcFtBR (list_):
    val = (list_[1]/100.)*(list_[4]/100)
    # gaussian error propagation:
    # A * B -> Add the relative uncertainties
    # they are uncorrelated -> quadratic sum
    un_u = np.sqrt( (list_[2]/list_[1])**2 + (list_[5]/list_[4])**2 )
    un_l = np.sqrt( (list_[3]/list_[1])**2 + (list_[6]/list_[4])**2 )
    # back to %
    val *= 100
    # and back to absolute uncertainties
    un_u *= val
    un_l *= val
    # return a list with the 
    return [list_[0], val, un_u, un_l]

In [43]:
c2e = 0.
c2e_u = 0.
c2e_l = 0.
for i in range(len(valuesHadrons2)):
    l = calcFtBR(valuesHadrons2[i])
    print(l)
    c2e += l[1]
    c2e_u += (l[2]**2)
    c2e_l += (l[3]**2)
c2e_u = np.sqrt(c2e_u)
c2e_l = np.sqrt(c2e_l)
print("c2e : ",np.round(c2e,2), "^{+",np.round(c2e_u,2),"}_{-",np.round(c2e_l,2),"}")
print("c2e : ",c2e, "+",c2e_u/c2e*100,"%-",c2e_l/c2e*100,"%")


['$D^{0}$', 2.53759, 0.20086719866618338, 0.26774051972012003]
['$D^{+}$', 2.7801100000000005, 0.4012452288812915, 0.44749408767044074]
['$D^{+}_{\\mathrm s}$', 0.47450000000000003, 0.14258292324117922, 0.1009449850165921]
['$\\Lambda^{+}_{\\mathrm c}$', 0.8058, 0.10830061172495747, 0.12363815147437299]
['$\\Xi^{0}_{\\mathrm c}$', 0.14400000000000002, 0.10820147873296372, 0.10746534325074295]
['$\\Xi^{+}_{\\mathrm c}$', 0.56, 0.37427396382863715, 0.371052556924218]
c2e :  7.3 ^{+ 0.62 }_{- 0.67 }
c2e :  7.302 + 8.499576224228948 %- 9.152452382728903 %


Now we can calculate the $c\bar{c} \rightarrow ee$ as c2e$^2$. For the uncertainties we solve:
\begin{equation}
\delta f(x,\delta x) := \sqrt{\left(\frac{\partial f(x)}{\partial x}\right)^{2}\delta x^{2}} 
\end{equation}
with $x^2$ we can wite:
\begin{equation}
\delta f(x,\delta x) := 2 \cdot x \cdot \delta x
\end{equation}

In [42]:
cc2ee = [c2e**2/100, (2* c2e * c2e_u)/100,  (2* c2e * c2e_l)/100]
print(np.round(cc2ee[0],2),"^{+",np.round(cc2ee[1],2),"}_{-",np.round(cc2ee[2],2),"}")

0.53 ^{+ 0.09 }_{- 0.1 }


This should now be compared with the value we have from the $Z^0 \rightarrow ee$ of $9.6\pm0.4$%.



In [9]:
cc2ee_old = [9.6**2/100, (2* 9.6 * 0.4)/100,  (2* 9.6 * 0.4)/100]
print(cc2ee_old)

[0.9216, 0.0768, 0.0768]


In [10]:
corrFac = cc2ee_old[0]/cc2ee[0]
print("The new points will be a factor", corrFac ,"higher")

The new points will be a factor 1.7284579117122607 higher


In [30]:
BRuncUp= cc2ee[1]/cc2ee[0]
BRuncDown= cc2ee[2]/cc2ee[0]
print("The uncertainty is now ",np.round(BRunc*100,1),"% instead of 22%")
print(np.round(corrFac,2),"^{}")

The uncertainty is now  18.3 % instead of 22%


In [31]:
# values from the publication
ypythia_ccbar  = [524., 854., 974.] 
ypythia_ccbar_sys  = [26.0, 145., 140.] 
ypythia_ccbar_stat = [61.0, 123., 138.] 
pythia_charm = [ypythia_ccbar,ypythia_ccbar_sys,ypythia_ccbar_stat]
ypowheg_ccbar = [756.0, 1251.0, 1417.0] 
ypowheg_ccbar_sys  = [38.00, 213.0, 204.0] 
ypowheg_ccbar_stat = [80.00, 155.0, 184.0]
powheg_charm = [ypowheg_ccbar,ypowheg_ccbar_sys,ypowheg_ccbar_stat]

# go through lists and multiply with correction factor
for x in range(len(pythia_charm)):
    for y in range(len(pythia_charm[x])):
        pythia_charm[x][y] = pythia_charm[x][y]*corrFac
for x in range(len(powheg_charm)):
    for y in range(len(powheg_charm[x])):
        powheg_charm[x][y] = powheg_charm[x][y]*corrFac


In [32]:
print("// # PYTHIA")
print("Double_t ypythia_ccbar[3]      = {",pythia_charm[0][0],",",pythia_charm[0][1],",",pythia_charm[0][2],"};")
print("Double_t ypythia_ccbar_sys[3]  = {",pythia_charm[1][0],",",pythia_charm[1][1],",",pythia_charm[1][2],"};")
print("Double_t ypythia_ccbar_stat[3] = {",pythia_charm[2][0],",",pythia_charm[2][1],",",pythia_charm[2][2],"};")
print("// # POWHEG")
print("Double_t ypowheg_ccbar[3]      = {",powheg_charm[0][0],",",powheg_charm[0][1],",",powheg_charm[0][2],"};")
print("Double_t ypowheg_ccbar_sys[3]  = {",powheg_charm[1][0],",",powheg_charm[1][1],",",powheg_charm[1][2],"};")
print("Double_t ypowheg_ccbar_stat[3] = {",powheg_charm[2][0],",",powheg_charm[2][1],",",powheg_charm[2][2],"};")

// # PYTHIA
Double_t ypythia_ccbar[3]      = { 905.7119457372246 , 1476.1030566022707 , 1683.5180060077419 };
Double_t ypythia_ccbar_sys[3]  = { 44.93990570451878 , 250.6263971982778 , 241.9841076397165 };
Double_t ypythia_ccbar_stat[3] = { 105.4359326144479 , 212.60032314060805 , 238.52719181629197 };
// # POWHEG
Double_t ypowheg_ccbar[3]      = { 1306.7141812544692 , 2162.3008475520382 , 2449.2248608962736 };
Double_t ypowheg_ccbar_sys[3]  = { 65.6814006450659 , 368.1615351947115 , 352.6054139893012 };
Double_t ypowheg_ccbar_stat[3] = { 138.27663293698086 , 267.91097631540043 , 318.03625575505595 };


In [38]:
# we can now calcualte the full uncertainty
print(np.round(pythia_charm[0][0],0),"\pm",np.round(pythia_charm[1][0],0),"(stat)\pm"
      ,np.round(pythia_charm[2][0]),"(syst)^{+"
      ,np.round(pythia_charm[0][0]*BRuncUp,0),"}_{-",np.round(pythia_charm[0][0]*BRuncDown,0),"}(BR)")
print(np.round(powheg_charm[0][0],0),"\pm",np.round(powheg_charm[1][0],0),"(stat)\pm"
      ,np.round(powheg_charm[2][0]),"(syst)^{+"
      ,np.round(powheg_charm[0][0]*BRuncUp,0),"}_{-",np.round(powheg_charm[0][0]*BRuncDown,0),"}(BR)")

906.0 \pm 45.0 (stat)\pm 105.0 (syst)^{+ 154.0 }_{- 166.0 }(BR)
1307.0 \pm 66.0 (stat)\pm 138.0 (syst)^{+ 222.0 }_{- 239.0 }(BR)
